**Versions:**

version 1:
- input user (only not processable allowed?)
- clean DataFrame
- KNN
- List as an output

version 2: 
- CLEAN UP THE ABSOLUTE MASSACER I COMMITED BY CHOSING SIZE AND AGE RANDOMLY FOR EVERY PET !!!!!!!!!!! (JP)
  - Size:
    - find out size for every dog-race
    - create new column with the size
  - Age:
   - compute birthday from "Date" and "Age upon Intake"
   - compute age from Birthday and today
   - recategorise it as puppy, teen, etc.
- geographical Data
- allow input which has to be processed (e.g. Dogsize -> Breed) ?
- allow a vet to regsiter a new pet to the DataFrame
- save the cleaned df 
- kick out rows that the user absolutley doesn´t want.
- ...

in the distant future:
- use github
- code a pretty interface (we don´t know how to do that haha!)
- ...

----------------------------------------------------


Documentation:



**1. DataFrame:**
 - Documentation:
  - creating column "Neutered" (Johanna)
  - replacing mixed colors with "mixed" (Thea)
  - the function returns the cleaned DataFrame (Julius)
  - creates a new column "Sex" 
  - makes up the personality traits 
  - makes up birthday ans size
  - renamed the columns so theyre the same as in the knn (jul)
 - Issues:

**2. Input_User:**
 - Documentation:
  - created it (Julius)
  - converted into int value array (JC)
  - empty values are now 0 (Julius)
  - matched the questions/allowed answers qith the DataFrame (julius)

 - Issues: 
  - must fit to KNN and DataFrame (Cave: Dogsizes, Age, Color: "Mixed")

**3. KNN:**
 - Documentation:
  - put everything in one function bc it breaks otherwise. df and vector get passed to it
  - copied to lines from jc´s prive code: pets/input_normalized

 - Issues:
  - if I understand it correctley we have to make sure that we don´t treat nominal data (e.g. the color) as ordinal data because then we´ll get skewed redults. \
  For example if we say {black: 1, white: 2, brown: 3} and the user preferes brown, but there are no brown pets, the KNN thinks white is better than black, because the number 2 is closer to 3 than the numer 1 ist to 3. \
  Maybe should change the conversion from string to integer so, that there are only to integer: {color user wants: 1, any other color: 0}. \
  That should solve that problem.
  - similarily: ordinal data has to be in the right order (e.g. age: puppy/kitten: 1, teen: 2, senoir: 3)
  - smiliarily np.nan shouldn´t be in there, because otherwise the KNN will prefere a attribute with is close (but only there randomly). I´d suggest we just delete the column if the user chose "don´t care" (Julius)

**4. Output:**
- prbly we can just do it in the KNN part bc its not much (Julius)


In [70]:
import numpy as np
import pandas as pd
import random
from scipy.spatial import distance

In [71]:
def clean_DataFrame():
  pd.read_csv?
  PetMatch_df = pd.read_csv('/content/sample_data/austin-animal-center-intakes-1.csv')
  del PetMatch_df['MonthYear']
  PetMatch_df.set_index('Animal ID', inplace = True)
  PetMatch_df.index.name = None

  # new data frame with split value columns 
  date = PetMatch_df["DateTime"].str.split(" ", n = 1, expand = True) 

  # making separate first name column from new data frame 
  PetMatch_df["Date"]= date[0] 

  # making separate last name column from new data frame 
  PetMatch_df["Time"]= date[1] 

  # Dropping old Name columns 
  PetMatch_df.drop(columns =["DateTime"], inplace = True) 

  PetMatch_df = PetMatch_df[['Name', 'Date', 'Time', 'Found Location', 'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake', 'Age upon Intake', 'Breed', 'Color',]]

  # adding a new column 'Neutered/Spayed' with value True or False
  PetMatch_df['Neutered/Spayed'] = np.where(PetMatch_df['Sex upon Intake'].str.contains('Spayed') | PetMatch_df['Sex upon Intake'].str.contains('Neutered'), 'Yes', 'No')
  PetMatch_df.loc[PetMatch_df['Sex upon Intake'] == 'Unknown', 'Neutered/Spayed'] = 0

  # adding column 'Sex', ugly but still working
  #PetMatch_df['Sex'] = 0
  PetMatch_df['Sex'] = np.where(PetMatch_df['Sex upon Intake'].str.contains('Female'), "Female", 'Male')
  PetMatch_df.loc[PetMatch_df['Sex upon Intake'] == 'Unknown', 'Sex'] = 0

  
  # rename columns 
  rename_dict = {
      'Animal Type' : 'Species',
      'Neutered/Spayed' : 'Neutered'
  }
  PetMatch_df.rename (columns = rename_dict, inplace = True)

  # make up columns for traits and size (only for dogs)
  traits_pool = {
    'Energy' : ['active', 'calm'],
    'Independence' : ['independent', 'dependent'],
    'Family' : ['extrovert', 'introvert'],
    'Size' : ['small', 'medium', 'big'],
    'Age_Dog' : ['puppy', 'teen', 'senior'],
  }

  for traits_group in traits_pool:
    char_value = random.choices(traits_pool[traits_group], k=107289)

    if traits_group == 'Size':
      PetMatch_df["Size"] = char_value
      PetMatch_df.loc[PetMatch_df['Species'] != 'Dog', 'Size'] = 0

    elif traits_group == 'Age_Dog':
      PetMatch_df["Age"] = char_value
      PetMatch_df.loc[((PetMatch_df['Species'] == 'Cat') &
                       (PetMatch_df['Age'] == 'puppy')),
                       'Age'] = 'kitten'

    else:
      PetMatch_df[traits_group] = char_value


  return PetMatch_df 
  

In [72]:
# Questions and Allowed Answers as Dictionary. 
# - Keys must be corresponding (capitalised!) to the index of the vector.
# - key_types must be a vector_key above in "questions" = already answered

questions = {
    "Species" : ["What species ?", ["Dog","Cat","dont care"]],
    "Age" : {
        "key_type" : "Species",
        "dog" : ["Which Age?", ["puppy","teen","senior","dont care"]],
        "cat" : ["Which Age?", ["kitten","teen","senior","dont care"]]
    },
    "Size" : {
        "key_type" : "Species",
        "dog" : ["Which Size", ["small","medium","big","dont care"]],
    },
    "Sex" : ["What sex?", ["Male","Female","dont care"]],
    "Neutered" : ["Neutered?", ["Yes","No","dont care"]],
    "Color" : ["Color?", ["White","Black","Brown", "Mixed", "dont care"]],
    "Energy" : ["full of energy or calm?", ["active","calm","dont care"]],
    "Family" : ["family friendly?", ["extrovert","introvert","dont care"]],
    "Independece" : ["independent?", ["independent","dependent","dont care"]]
}

In [73]:
# handles the interaction with the user
def ask_user():


  # creates an empty Series with "criteria" as index. Only change "criteria", the Series expands automatically 
  criteria = [key for key in questions]

  dummies = np.zeros([len(criteria)])
  vector = pd.Series(dummies, index=criteria)


  # retrieves the needed stringd from the dictionary
  def search_questions(dictionary, my_key):
    
    try:
        
        subset = dictionary[my_key]
        
        # check whether a sub-dictionary to reach the maximla depth of he dict
        if isinstance(subset, dict):
        
            #retrieves the user input of the needed criterium from the vector
            key_type = dictionary[my_key]["key_type"]
            new_key = vector.loc[key_type]
            
            # calls itself with the sub_dict and needed key. Returns itself bc the else part is always triggered
            return search_questions(subset, new_key)
    
        else:
            
            # the subset is a list. It is returned up through the entire recursion
            return subset
        
    # if there exists no entry for a (specific) attribute, False is returned
    except KeyError:
        return False


  # asks the chosen questons and allowed answers
  def ask_input(vector_key, ques_allo):
    
    # if there exists no attribute -> "ques_allo" = False -> the function is skipped
    if ques_allo:
        
        ques, allo = ques_allo[0], ques_allo[1]
        
        user_input = str(input(str(ques)+ str(allo)))
        
        # if the user_input is not allowed the function calls itself
        if user_input not in allo:
            print("Please only one of the possible answers !")
            ask_input(vector_key, ques_allo)
            
        # the user_input ist saved in the vector
        else:
            vector.loc[vector_key] = user_input

  # replaces everything not equal to these terms with 'Mixed'
  
  PetMatch_df = clean_DataFrame()
  
  PetMatch_df.loc[((PetMatch_df['Color'] != 'White')&
                  (PetMatch_df['Color'] != 'Black')&
                  (PetMatch_df['Color'] != 'Brown')&
                  (PetMatch_df['Color'] != 'Gray')&
                  (PetMatch_df['Color'] != 'Orange'))
                  ,'Color'] = 'Mixed'


  # main algo: fills the vector with the user´s answers
  for vector_key in vector.index:
      ques_allo = search_questions(questions, vector_key)
      ask_input(vector_key, ques_allo)

  return vector
  #user_input = vector.to_numpy()

  #user_input = ask_user()


    

In [74]:
################### CREATION OF DICTIONARIES
#get all unique entries for columns in dataframe 'animals' and store unique 
#entries in arrays, so that they can be assigned numeric values later

def createDict_PetMatch_df(PetMatch_clean, user_input):

  Species_uni = PetMatch_clean.Species.unique()
  Size_uni = PetMatch_clean.Size.unique()
  Age_uni = PetMatch_clean.Age.unique()
  Sex_uni = PetMatch_clean.Sex.unique()
  Neutered_uni = PetMatch_clean.Neutered.unique()
  Color_uni = PetMatch_clean.Color.unique()
  Energy_uni = PetMatch_clean.Energy.unique()
  Family_uni = PetMatch_clean.Family.unique()
  Independence_uni = PetMatch_clean.Independence.unique()

#assign numeric value to each unique entries to arrays ['column name']_uni
#create dict for columns, set to 0

#for column 'Species'
  Kspecies = dict()
  species_num = 0
  for species in Species_uni:
    Kspecies[species] =  species_num + 1
    species_num = species_num + 1


#for column 'Size'
  Ksize = dict()
  size_num = 0
  for size in Size_uni:
    Ksize[size] = size_num + 1
    size_num = size_num + 1  


#for column 'Age'
  Kage = dict()
  age_num = 0
  for age in Age_uni:
    Kage[age] =  age_num + 1
    age_num = age_num + 1


#for column 'Sex'
  Ksex = dict()
  sex_num = 0
  for sex in Sex_uni:
    Ksex[sex] = sex_num + 1
    sex_num = sex_num + 1


#for column Neutered
  Kneutered = dict()
  neutered_num = 0
  for neutered in Neutered_uni:
    Kneutered[neutered] = neutered_num + 1
    neutered_num = neutered_num + 1


#for column Color
  Kcolor = dict()
  color_num = 0
  for color in Color_uni:
    Kcolor[color] = color_num + 1
    color_num = color_num + 1


#for column Energy
  Kenergy = dict()
  energy_num = 0
  for energy in Energy_uni:
     Kenergy[energy] = energy_num + 1
     energy_num = energy_num + 1
  

#for column Family
  Kfamily = dict()
  family_num = 0
  for family in Family_uni:
    Kfamily[family] = family_num + 1
    family_num = family_num + 1


#for column Independence
  Kindependence = dict()
  independence_num = 0
  for independence in Independence_uni:
    Kindependence[independence] = independence_num + 1
    independence_num = independence_num + 1

################### END CREATION DICTIONARIES
################### REPLACE COLUMN STRING VALUES WITH NUMERIC VALUES FROM DICTIONARIES

#Create list of old values from dict keys (e.g. "species") and create list 
#of new values from dict values to use to replace strings in original 
#dataframe columns (e.g. replace "cat" in dataframe column with numeric value "1")

  Kspecies_Keylist = list(Kspecies.keys())
  Kspecies_Valuelist = list(Kspecies.values())

  Ksize_Keylist = list(Ksize.keys())
  Ksize_Valuelist = list(Ksize.values())

  Kage_Keylist = list(Kage.keys())
  Kage_Valuelist = list(Kage.values())

  Ksex_Keylist = list(Ksex.keys())
  Ksex_Valuelist = list(Ksex.values())

  Kneutered_Keylist = list(Kneutered.keys())
  Kneutered_Valuelist = list(Kneutered.values())

  Kcolor_Keylist = list(Kcolor.keys())
  Kcolor_Valuelist = list(Kcolor.values())

  Kenergy_Keylist = list(Kenergy.keys())
  Kenergy_Valuelist = list(Kenergy.values())

  Kfamily_Keylist = list(Kfamily.keys())
  Kfamily_Valuelist = list(Kfamily.values())

  Kindependence_Keylist = list(Kindependence.keys())
  Kindependence_Valuelist = list(Kindependence.values())

#replace original column values with numeric values in unique array-list
 
  PetMatch_clean_num = pd.DataFrame(PetMatch_clean, columns = ['Species', 'Size', 'Age', 'Sex', 'Neutered', 'Color', 'Energy', 'Family', 'Independence'])

  PetMatch_clean_num['Species'] = PetMatch_clean['Species'].replace(Kspecies_Keylist, Kspecies_Valuelist)
  PetMatch_clean_num['Size'] = PetMatch_clean['Size'].replace(Ksize_Keylist, Ksize_Valuelist)
  PetMatch_clean_num['Age'] = PetMatch_clean['Age'].replace(Kage_Keylist, Kage_Valuelist)
  PetMatch_clean_num['Sex'] = PetMatch_clean['Sex'].replace(Ksex_Keylist, Ksex_Valuelist)
  PetMatch_clean_num['Neutered'] = PetMatch_clean['Neutered'].replace(Kneutered_Keylist, Kneutered_Valuelist)
  PetMatch_clean_num['Color'] = PetMatch_clean['Color'].replace(Kcolor_Keylist, Kcolor_Valuelist)
  PetMatch_clean_num['Energy'] = PetMatch_clean['Energy'].replace(Kenergy_Keylist, Kenergy_Valuelist)
  PetMatch_clean_num['Family'] = PetMatch_clean['Family'].replace(Kfamily_Keylist, Kfamily_Valuelist)
  PetMatch_clean_num['Independence'] = PetMatch_clean['Independence'].replace(Kindependence_Keylist, Kindependence_Valuelist)

  ################### END REPLACE STRING COLUMN VALUES WITH NUMERIC VALUES FROM DICTIONARIES

  ##################### replace str values in User Input vector with key-val values in criteria dictionaries


  dict_crit = [Kspecies, Ksize, Kage, Ksex, Kneutered, Kcolor, Kenergy, Kfamily, Kindependence]

  user_input_num = user_input.copy()


#Replace NaN values with 0 in case of "dont care" or cat decision tree
# np.nan_to_num(0, copy=True, nan=0, posinf=None, neginf=None)
#User_input_num = np.nan_to_num(0, copy=True)


  Uinput_index = 0
  for Input in user_input_num: #dog
    for dictionary in dict_crit: #Kspecies
      if Input in dictionary.keys(): #True
        user_input_num[Uinput_index] = int(dictionary[Input]) #2
    Uinput_index = Uinput_index + 1
  print(user_input_num)#-------------------------------------------------------------testing
  user_input_num.astype(int)

############ END CALCULATE EUCLIDEAN DISTANCES AND ADD "EUCLIDEAN DISTANCES" AND ADD COLUMN TO DF "ANIMALS" FOR READABILITY

  PetMatch_norm = PetMatch_clean_num.copy()
  userInput_norm = user_input_num.copy()
  
  #PetMatch_norm = (PetMatch_norm - np.min(PetMatch_norm)) / (np.max(PetMatch_norm) - np.min(PetMatch_norm))
  #userInput_norm = (user_input_num - np.min(PetMatch_norm)) / (np.max(PetMatch_norm) - np.min(PetMatch_norm))


############ CALCULATE EUCLIDEAN DISTANCES AND ADD "EUCLIDEAN DISTANCES" AND ADD COLUMN TO DF "ANIMALS" FOR READABILITY

#def Calc_EuclidDistance ():  
# Find the distance between User Input normalized vector and rows of df animals_num.
  euclidean_distances = PetMatch_norm.apply(lambda row: distance.euclidean(row, userInput_norm), axis=1)


#make copy of df animals and save as "animals_euclid"
  PetMatch_euclid = PetMatch_clean.copy()

#insert column "Euclidean distance" in position 9 in df animals
  PetMatch_euclid.insert(9, 'Euclidean distances', euclidean_distances)

  PetMatch_eSorted = PetMatch_euclid.copy()

  PetMatch_eSorted = PetMatch_eSorted.sort_values(by='Euclidean distances', ascending=True)
  print(euclidean_distances)
  return PetMatch_eSorted
  
 

############ END CALCULATE EUCLIDEAN DISTANCES AND ADD "EUCLIDEAN DISTANCES" AND ADD COLUMN TO DF "ANIMALS" FOR READABILITY

In [75]:
# Part 1 : Create a cleaned DataFrame
PetMatch_df = clean_DataFrame()

PetMatch_clean = PetMatch_df[['Species', 'Color','Neutered', 'Sex', 'Energy', 'Independence','Age', 'Family', 'Size']]
  
# replaces everything not equal to these terms with 'Mixed'
PetMatch_clean.loc[((PetMatch_clean['Color'] != 'White')&
                 (PetMatch_clean['Color'] != 'Black')&
                 (PetMatch_clean['Color'] != 'Brown')&
                 (PetMatch_clean['Color'] != 'Gray')&
                 (PetMatch_clean['Color'] != 'Orange'))
                 ,'Color'] = 'Mixed'

# Part 2 : Create a interface for the user and return the input
user_input = ask_user()
# user_input = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]) #---------------------------------------TESSTING

# Part 3: KNN

res = createDict_PetMatch_df(PetMatch_clean, user_input)

Final_out = res.loc[res['Species'] == user_input[0]]




/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


What species ?['Dog', 'Cat', 'dont care']Cat
What sex?['Male', 'Female', 'dont care']Female
Neutered?['Yes', 'No', 'dont care']Yes
Color?['White', 'Black', 'Brown', 'Mixed', 'dont care']White
full of energy or calm?['active', 'calm', 'dont care']active
family friendly?['extrovert', 'introvert', 'dont care']introvert
independent?['independent', 'dependent', 'dont care']dependent
Species        2
Age            1
Size           1
Sex            3
Neutered       3
Color          4
Energy         1
Family         1
Independece    2
dtype: object
A803421    4.358899
A803419    3.872983
A800462    4.242641
A800463    4.582576
A803416    4.358899
             ...   
A664233    4.000000
A664237    6.082763
A664235    6.000000
A664236    6.000000
A521520    3.316625
Length: 107289, dtype: float64


In [76]:
print(user_input)
Final_out

Species              Cat
Age                    0
Size                   0
Sex               Female
Neutered             Yes
Color              White
Energy            active
Family         introvert
Independece    dependent
dtype: object


,Species,Color,Neutered,Sex,Energy,Independence,Age,Family,Size,Euclidean distances
A689813,Cat,White,Yes,Female,active,dependent,teen,introvert,0,3.000000
A716823,Cat,White,Yes,Female,active,dependent,teen,introvert,0,3.000000
A743082,Cat,White,Yes,Female,active,dependent,teen,introvert,0,3.000000
A740628,Cat,White,Yes,Female,active,dependent,teen,introvert,0,3.000000
A776164,Cat,White,Yes,Female,active,dependent,senior,introvert,0,3.162278
...,...,...,...,...,...,...,...,...,...,...
A759861,Cat,Mixed,0,0,calm,independent,kitten,extrovert,0,6.164414
A725662,Cat,Mixed,0,0,calm,independent,kitten,extrovert,0,6.164414
A706099,Cat,Mixed,0,0,calm,independent,kitten,extrovert,0,6.164414
A750879,Cat,Mixed,0,0,calm,independent,kitten,extrovert,0,6.164414
